In [1]:
import sys
sys.path.append("../")
import numpy as np
import time
import pandas as pd
import pickle
import math
from typing import Tuple


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from scipy.sparse import hstack

ANALYSIS_POSTFIX = "mined_sudden_2024-08-26"

experiment_config = {
    "RS" : 42,
    "ANALYSIS_POSTFIX": ANALYSIS_POSTFIX
}

In [2]:
def step_two(experiment_config, 
             X_train,
             y_train,
             model,
             X_val=None,
             y_val=None,
             save=False): 
    
    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]
    
    training_start_time = time.time()
    if model=="lr":
        reg = LinearRegression().fit(X_train, y_train)
    elif model =="svm": 
        reg = SVR().fit(X_train, y_train)
    elif model=="rf":
        reg = RandomForestRegressor.fit(X_train, y_train)
    elif model=="lgbm":
        reg = LGBMRegressor(max_depth=10, silent=True)
        reg.fit(X=X_train, y=y_train)
    elif model=="catboost":
        reg = CatBoostRegressor()
        reg.fit(X=X_train, y=y_train)
    training_end_time = time.time()
    time_training = training_end_time - training_start_time

    

    if save:
        with open(f'./models/reg_{model}_{ANALYSIS_POSTFIX}.pkl','wb') as f:
            pickle.dump(reg, f)
        return f'./models/reg_{model}_{ANALYSIS_POSTFIX}.pkl'
    
    else:
        inference_start_time = time.time()
        y_pred = reg.predict(X_val)
        inference_end_time = time.time()
        time_inference = inference_end_time - inference_start_time

        y_pred[y_pred<0] = 0
        mae = mean_absolute_error(y_true=y_val, y_pred=y_pred)
        rmse = math.sqrt(mean_squared_error(y_true=y_val, y_pred=y_pred))
        return {"pred": y_pred, "mae": mae, "rmse": rmse, "time_training" : time_training, "time_inference" : time_inference}
    

def cv_step_2(experiment_config:dict, cv_df:pd.DataFrame) -> Tuple:

    t_models = ["lr", "svm", "lgbm", "catboost"]

    results = {}


    for test_fold in range(cv_df.fold.max()+1):
        print(test_fold)

        # Prepare the input data
        vectorizer = TfidfVectorizer()
        X_train_tfidf = vectorizer.fit_transform(cv_df.loc[cv_df.fold!=test_fold, "input_sequence"])
        X_train_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold!=test_fold, "model_set"], sparse=True).sparse.to_coo().tocsr()
        X_train = hstack([X_train_column_sparse, X_train_tfidf])
        y_train = cv_df.loc[cv_df.fold!=test_fold, "rouge"]
        
        X_val_tfidf = vectorizer.transform(cv_df.loc[cv_df.fold==test_fold, "input_sequence"])
        X_val_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold==test_fold, "model_set"], sparse=True).sparse.to_coo().tocsr()
        X_val = hstack([X_val_column_sparse, X_val_tfidf])
        y_val = cv_df.loc[cv_df.fold==test_fold, "rouge"]

        results[test_fold] = {}
        for model in t_models:
            print(model)
            preds_df = step_two(experiment_config=experiment_config,
                                X_train=X_train,
                                y_train=y_train,
                                X_val=X_val,
                                y_val=y_val,
                                model=model)
            cv_df.loc[cv_df.fold==test_fold, f"{model}_perf_hat"] = preds_df["pred"]
            results[test_fold][model] = preds_df

    cv_df = cv_df.reset_index(drop=True)

    # rearrange results
    model_results = {}

    for model in t_models:
        model_results[model]= {}
        model_results[model]["rmse"] = []
        model_results[model]["mae"] = [] 

        for fold in range(3):
        
            model_results[model]["mae"].append(results[fold][model]["mae"])
            model_results[model]["rmse"].append(results[fold][model]["rmse"])
        
        model_results[model]["rmse_avg"] = np.array(model_results[model]["rmse"]).mean()
        model_results[model]["mae_avg"] = np.array(model_results[model]["mae"]).mean()

        model_results[model]["rmse_std"] = np.array(model_results[model]["rmse"]).std()
        model_results[model]["mae_std"] = np.array(model_results[model]["mae"]).std()

    for model in t_models:
        print(model)
        print("RMSE ", model_results[model]["rmse_avg"])
        print("MAE ",model_results[model]["mae_avg"])
        print("\n")

        print("RMSE STD ", model_results[model]["rmse_std"])
        print("MAE STD",model_results[model]["mae_std"])
        print("\n")

    return cv_df, model_results

def full_step_2(cv_df:pd.DataFrame,
                experiment_config:dict) -> None:
    
    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]
    # TRAIN ON ALL PREDICTIONS AT ONCE

    t_models = ["lr", "svm", "lgbm", "catboost"]

    # Prepare the input data
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(cv_df.loc[cv_df.model_set!="ensemble", "input_sequence"])
    X_train_column_sparse = pd.get_dummies(cv_df.loc[cv_df.model_set!="ensemble", "model_set"], sparse=True).sparse.to_coo().tocsr()
    X_train = hstack([X_train_column_sparse, X_train_tfidf])
    y_train = cv_df.loc[cv_df.model_set!="ensemble", "rouge"]
        
    with open(f"./models/vectorizer_{ANALYSIS_POSTFIX}.pkl", "wb") as file:
        pickle.dump(vectorizer, file, protocol=pickle.HIGHEST_PROTOCOL) 
        
    for model in t_models:
        print(model)
        preds_df = step_two(experiment_config=experiment_config,
                            X_train=X_train,
                            y_train=y_train,
                            model=model,
                            save=True)

In [3]:
with open(f"../ensemble_learning/reports/results/{ANALYSIS_POSTFIX}/cv_results.pickle", "rb") as handle:
    cv_predictions = pickle.load(handle)

with open(f"../ensemble_learning/reports/results/{ANALYSIS_POSTFIX}/test_results.pickle", "rb") as handle:
    test_predictions = pickle.load(handle)


In [4]:
COLUMNS_TEST = ['question_id', 'parent_answer_post_id', 'prob', 'input_sequence',
       'output_sequence', 'id', 'snippet_len', 'intent_len', 'snippet_token_n',
       'intent_token_n', 'cluster', 'input_ids', 'attention_mask', 'labels',
       'prediction', 'rouge', 'model_set']

COLUMNS_CV = COLUMNS_TEST.copy()
COLUMNS_CV.append("fold")

#### Preprocessing

In [5]:
cv_predictions = cv_predictions.loc[cv_predictions.model_set!="ensemble", COLUMNS_CV]
test_predictions = test_predictions.loc[cv_predictions.model_set!="ensemble", COLUMNS_TEST]

# Code Only

We have 9 base lerner settings models that we compare learning of 1, splitting to two meta models,  all together. 

In [6]:
MODELS_LIST = [0, 1, 2, 5, 10, 'cluster_[1]', 'cluster_[4]', 'cluster_[3]', 'cluster_[0, 1, 4]']
MODE = ["ONE-BY-ONE", "TWO-MODELS", "ALL"]

In [ ]:
for model_base in MODELS_LIST:

    temp_df =  cv_predictions.loc[cv_predictions.model_set==model_base]
    temp_df, model_results = cv_step_2(experiment_config=experiment_config,
              cv_df=temp_df)
    
    
    
    break

In [30]:
results_df = pd.DataFrame()

t_models = ["lr", "svm", "lgbm", "catboost"]

for model in t_models:
    for cluster in sorted(temp_df.cluster.unique()):

        print(cluster)
        cluster_temp_df = temp_df.loc[temp_df.cluster==cluster, :]

        folds_mae, folds_rmse = [], []
        
        for fold in range(3):
            mae = mean_absolute_error(y_true=cluster_temp_df.loc[cluster_temp_df.fold==fold, "rouge"],
                                      y_pred=cluster_temp_df.loc[cluster_temp_df.fold==fold, f"{model}_perf_hat"])
            folds_mae.append(mae)
            
            rmse = math.sqrt(mean_squared_error(y_true=cluster_temp_df.loc[cluster_temp_df.fold==fold, "rouge"],
                                      y_pred=cluster_temp_df.loc[cluster_temp_df.fold==fold, f"{model}_perf_hat"]))
            folds_rmse.append(rmse)

        mae = mean_absolute_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                      y_pred=cluster_temp_df.loc[:, f"{model}_perf_hat"])
        
        rmse = math.sqrt(mean_squared_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                      y_pred=cluster_temp_df.loc[:, f"{model}_perf_hat"]))
        
        t_res = pd.DataFrame(data={"model_meta": model, "cluster": cluster, "rmse_mean": rmse, "rmse_std": np.array(folds_rmse).std(), "mae_mean": mae, "mae_std": np.array(folds_mae).std()}, index=[0])


        results_df = pd.concat([results_df, t_res], axis=0)
    


for model in t_models:

    folds_mae, folds_rmse = [], []
    
    for fold in range(3):
        mae = mean_absolute_error(y_true=temp_df.loc[temp_df.fold==fold, "rouge"],
                                    y_pred=temp_df.loc[temp_df.fold==fold, f"{model}_perf_hat"])
        folds_mae.append(mae)
        
        rmse = math.sqrt(mean_squared_error(y_true=temp_df.loc[temp_df.fold==fold, "rouge"],
                                    y_pred=temp_df.loc[temp_df.fold==fold, f"{model}_perf_hat"]))
        folds_rmse.append(rmse)

    mae = mean_absolute_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model}_perf_hat"])
    
    rmse = math.sqrt(mean_squared_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model}_perf_hat"]))
    
    t_res = pd.DataFrame(data={"model_meta": model, "cluster": "full", "rmse_mean": rmse, "rmse_std": np.array(folds_rmse).std(), "mae_mean": mae, "mae_std": np.array(folds_mae).std()}, index=[0])


    results_df = pd.concat([results_df, t_res], axis=0)


results_df = results_df.sort_values(["model_meta", "cluster"])

0
1
2
3
4
0
1
2
3
4
0
1
2
3
4
0
1
2
3
4


In [31]:
results_df





,model,cluster,rmse_mean,rmse_std,mae_mean,mae_std
0,catboost,0,0.112115,0.006296,0.094472,0.005743
0,catboost,1,0.112536,0.016195,0.091059,0.011971
0,catboost,2,0.130640,0.002089,0.105216,0.000883
0,catboost,3,0.144274,0.003632,0.114903,0.002323
0,catboost,4,0.125793,0.002013,0.101976,0.000844
0,catboost,full,0.133906,0.001316,0.107373,0.000185
0,lgbm,0,0.112056,0.006191,0.092570,0.005152
0,lgbm,1,0.111786,0.010189,0.092765,0.007869
0,lgbm,2,0.130136,0.003038,0.104743,0.001400
0,lgbm,3,0.143615,0.004118,0.115004,0.003336
